# DETECTING AND GENERATING OF IMAGE DATA

In [1]:
import cv2
import numpy as np
from PIL import Image
import os

In [2]:
facecascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
def generate_dataset(image, p_id, image_id):
    cv2.imwrite('face_data/user.'+str(p_id)+'.'+str(image_id)+'.jpg',image)

In [4]:
def draw_boundary_1(image, classifier, scalefactor, min_neighbors, color, text):
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_scale, scalefactor, min_neighbors)
    coordinates = []
    
    for (x, y, w, h) in features:
        cv2.rectangle(image, (x,y),(x+w, y+h), color, 1)        
        cv2.putText(image, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
        coordinates = [x, y, w, h]
        
    return coordinates
    

In [5]:
def detect_1(image, facecascade, image_id):
    color = {'blue':(0,0,255), 'red':(255,0,0), 'green':(0,255,0), 'white':(255,255,255)}
    coordinate = draw_boundary_1(image, facecascade,1.1, 10, color['white'], 'FACE')
    
    if len(coordinate)==4:
        area_of_focus = image[coordinate[1]:coordinate[1]+coordinate[3], coordinate[0]:coordinate[0]+coordinate[2]]
               
        p_id = 1
        generate_dataset(area_of_focus, p_id, image_id)
    return image

In [6]:
def detect_2(image, facecascade, image_id):
    color = {'blue':(0,0,255), 'red':(255,0,0), 'green':(0,255,0), 'white':(255,255,255)}
    coordinate = draw_boundary_1(image, facecascade,1.1, 10, color['red'], 'FACE')
    
    if len(coordinate)==4:
        area_of_focus = image[coordinate[1]:coordinate[1]+coordinate[3], coordinate[0]:coordinate[0]+coordinate[2]]
               
        p_id = 2
        generate_dataset(area_of_focus, p_id, image_id)
    return image

In [7]:
video_capture  = cv2.VideoCapture(0) #if using an external webcam type in -1 inplace of 0
image_id = 0

while True:
    _ ,image = video_capture.read()
    image = detect_1(image, facecascade, image_id)
    #image = recognize(image, clf, facecascade)
    cv2.imshow('Face Detection', image)
    image_id += 1
    if cv2.waitKey(1) & 0xFF == ord('e'): # to break loop press e
        break
        
video_capture.release()
cv2.destroyAllWindows()

In [9]:
video_capture  = cv2.VideoCapture(0) #if using an external webcam type in -1 inplace of 0
image_id = 0

while True:
    _ ,image = video_capture.read()
    image = detect_2(image, facecascade, image_id)
    #image = recognize(image, clf, facecascade)
    cv2.imshow('Face Detection', image)
    image_id += 1
    if cv2.waitKey(1) & 0xFF == ord('e'): # to break loop press e
        break
        
video_capture.release()
cv2.destroyAllWindows()

# IMAGE TRAINING

In [12]:
def train_classifier(data_dir):
        path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
        faces = []
        ids = []
        
        for image_ in path:
            img = Image.open(image_).convert('L')
            image_numpy = np.array(img, 'uint8')
            #facez = classifier.detectMultiScale(image_numpy, scalefactor, min_neighbors)
            id_ = int(os.path.split(image_)[1].split(".")[1])
            
            faces.append(image_numpy)
            ids.append(id_)
            
        ids= np.array(ids)
        
        clf = cv2.face.LBPHFaceRecognizer_create()
        clf.train(faces, ids)
        clf.write('classifier.yml')
            
train_classifier('face_data')

# ---------------------------------------------------------------------------------------------------------------

# IMAGE DETECTION/PREDICTION

In [16]:
facecascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read('classifier.yml')

In [19]:
def draw_boundary(image, classifier, scalefactor, min_neighbors, color, text, clf):
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_scale, scalefactor, min_neighbors)
    coordinates = []
    
    for (x, y, w, h) in features:
        cv2.rectangle(image, (x,y),(x+w, y+h), color, 1)
        id_, _= clf.predict(gray_scale[y:y+h, x:x+w])
        
        if id_ == 1:
            cv2.putText(image, 'xxxx', (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
            coordinates = [x, y, w, h]
        elif id_ == 2 :
            cv2.putText(image, 'yyyy', (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
            coordinates = [x, y, w, h]
        
        
        
        
    return coordinates
    

In [20]:
def recognize(image, clf,facecascade ):
    color = {'blue':(0,0,255), 'red':(255,0,0), 'green':(0,255,0), 'white':(255,255,255)}
    coordinate = draw_boundary(image,facecascade,1.1, 10, color['white'], 'FACE', clf)
    return image
    

In [21]:
video_capture  = cv2.VideoCapture(0) #if using an external webcam type in -1 inplace of 0
image_id = 0

while True:
    _ ,image = video_capture.read()
    #image = detect(image, facecascade, image_id)
    image = recognize(image, clf, facecascade)
    cv2.imshow('Face Detection', image)
    image_id += 1
    if cv2.waitKey(1) & 0xFF == ord('e'): # to break loop press e
        break
        
video_capture.release()
cv2.destroyAllWindows()